In [10]:
import numpy as np
from pandas import Series, DataFrame
import pandas as pd
from pandas import read_html

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

In [11]:
#Constants for accessing DataFrame data pulled from web
QB = 0
RB = 1
WR = 2
TE = 3
DEF = 4
KICKER = 5

In [12]:
#Set to create unique in house DataFrame 

tup = [('', 'Bye'),
       ('', 'Pts'),
       ('Passing', 'Att'),
       ('Passing', 'Cmp'),
       ('Passing', 'Yds'),
       ('Passing', 'TD'),
       ('Passing', 'Int'),
       ('Passing', '2Pt'),
       ('Rushing', 'Att'),
       ('Rushing', 'Yds'),
       ('Rushing', 'TD'),
       ('Rushing', '2Pt'),
       ('Receiving', 'Rec'),
       ('Receiving', 'Yds'),
       ('Receiving', 'TD'),
       ('Receiving', '2Pt'),
       ('Fumbles', 'FL'),
       ('Fumbles', 'TD')
      ]

In [13]:
#Set to change the index names 

indexDEF = ['Bye', 'Pts*', 'Sack', 'Int', 'Saf', 'FR', 'Blk', 'TD', 'PA',
       'PassYds/G', 'RushYds/G', 'TotYds/G']

indexKicker = ['Bye', 'Pts*', 'XPA', 'XPM', 'FGA', 'FGM', '50+']


In [14]:
#Load data from web for 2017 season
def loadWebData(frameArray):
    url_qb_season17 = 'https://www.footballdb.com/fantasy-football/index.html?pos=QB&yr=2017&wk=all&rules=1'
    url_rb_season17 = 'https://www.footballdb.com/fantasy-football/index.html?pos=RB&yr=2017&wk=all&rules=1'
    url_wr_season17 = 'https://www.footballdb.com/fantasy-football/index.html?pos=WR&yr=2017&wk=all&rules=1'
    url_te_season17 = 'https://www.footballdb.com/fantasy-football/index.html?pos=TE&yr=2017&wk=all&rules=1'
    url_def_season17 = 'https://www.footballdb.com/fantasy-football/index.html?pos=DST&yr=2017&wk=all&rules=1'
    url_kicker_season17 = 'https://www.footballdb.com/fantasy-football/index.html?pos=K&yr=2017&wk=all&rules=1'
    

    frameArray.insert(QB, pd.io.html.read_html(url_qb_season17)[0])
    frameArray.insert(RB, pd.io.html.read_html(url_rb_season17)[0])
    frameArray.insert(WR, pd.io.html.read_html(url_wr_season17)[0])
    frameArray.insert(TE, pd.io.html.read_html(url_te_season17)[0])
    frameArray.insert(DEF, pd.io.html.read_html(url_def_season17)[0]) 
    frameArray.insert(KICKER, pd.io.html.read_html(url_kicker_season17)[0]) 
    
    
    
    
    
    
    

In [15]:
#Fix name that contains the next line string '\xa0'
#nameIndex: Empty index that will contain the fixed name
#data: Pandas Series type that contains the names of each player
def nameFix(nameIndex, data):
    #Get name from Player column and save to nameIndex
    NAME_FIX_OFFSET = 3

    for i in data:
        #print(i)
        nameIndex.append(i)


    size = len(nameIndex)

    for i in range(size):
        a1 = nameIndex[i]
        count = 0

        for x in a1:
            count += 1
            if x == u'\xa0':
                break

        count -= NAME_FIX_OFFSET
        nameIndex[i] = a1[:count]
        
 

In [16]:
def setDataFrame(dframe, dataType, index):
    nameIndex = []
    dataMatrix = []
    
    shape = dframe.shape
    shape[0]
    
    for count in range(shape[0]):
        temp = dframe.iloc[count].iloc[1:]
        a1 = []
        for i in temp:
            a1.append(i)

        dataMatrix.append(a1)
  
    dataMatrix = np.array(dataMatrix)
    nameFix(nameIndex, frameArray[dataType]['\xa0'].get("Player"))  
    
    return DataFrame(dataMatrix, columns = index, index = nameIndex)


In [17]:
def renameIndex(dframe, teamName):

    d1 = dict()
    index = 0

    for i in teamName:
        d1[index] = i
        index += 1
        
    dframe = (dframe.rename(d1, axis = 'index'))
    return(dframe)
    

In [18]:
def trimName(teamName):    
    arryIndex = len(teamName)
    
    
    for a1 in range(arryIndex):
        #a1 is index for the string array teamName 
        strLen = len(teamName.iloc[a1])
        for b1 in teamName.iloc[a1][::-1]:
            if b1.isupper():
                strLen -= 1
            else:
                break
                
        teamName.iat[a1] = (teamName.iloc[a1][:strLen])
       
    return teamName

In [19]:
frameArray = []
index = pd.MultiIndex.from_tuples(tup)
loadWebData(frameArray)

In [39]:
frameQB = setDataFrame(frameArray[QB], QB, index)
frameRB = setDataFrame(frameArray[RB], RB, index)
frameWR = setDataFrame(frameArray[WR], WR, index)
frameTE = setDataFrame(frameArray[TE], TE, index)


In [40]:
teamName = frameArray[DEF].get('Team')
teamName = trimName(teamName)

#Remove 'Team' column from original DataFrame. Create rename index with team names
frameDef = DataFrame(frameArray[DEF], columns = indexDEF )
frameDef = renameIndex(frameDef, teamName)
frameDef = frameDef.rename({'Pts*': 'Pts'}, axis='columns')


In [41]:
nameIndex = []
teamName = frameArray[KICKER].get('Player')
nameFix(nameIndex, teamName) 

frameKicker = DataFrame(frameArray[KICKER], columns = indexKicker)
frameKicker = renameIndex(frameKicker, nameIndex)
frameKicker = frameKicker.rename({'Pts*': 'Pts'}, axis='columns')



In [47]:
frameKicker

,Bye,Pts,XPA,XPM,FGA,FGM,50+
Greg Zuerlein,8,170.0,46,44,40,38,6
Stephen Gostkowski,9,164.0,47,45,40,37,4
Robbie Gould,11,153.0,30,28,41,39,4
Matt Bryant,0,153.0,35,35,39,34,8
Justin Tucker,10,151.0,39,39,37,34,5
Harrison Butker,10,150.0,28,28,42,38,4
Chris Boswell,9,150.0,39,37,38,35,4
Wil Lutz,0,148.0,50,47,36,31,4
Matt Prater,7,144.0,41,40,35,30,7
Kai Forbath,9,142.0,39,34,38,32,6
